In [1]:
# workflow for single book
from nlppln import WorkflowGenerator
cwl_working_dir = '/home/dafne/cwl-working-dir/'

In [2]:
with WorkflowGenerator(working_dir=cwl_working_dir) as wf_sub:
    wf_sub.load(steps_dir='../adhtools/cwl/')
    wf_sub.load(steps_dir='../java/cwl/')
    print(wf_sub.list_steps())
    
    analyzer = wf_sub.add_input(analyzer='string', default='Alkhalil')
    book = wf_sub.add_input(book='File')
    cp = wf_sub.add_input(cp='string')
    
    metadata, txt_dir = wf_sub.txt2safar_input(in_file=book)
    analyzed_files = wf_sub.SafarAnalyze(in_dir=txt_dir, analyzer=analyzer, cp=cp)
    safar_output_dir = wf_sub.safar_add_metadata(in_files=analyzed_files, meta_in=metadata)
    
    wf_sub.add_outputs(safar_output_dir=safar_output_dir)
    
    wf_sub.save('../adhtools/cwl/safar-workflow.cwl', wd=True, relative=False)

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-03-27 10:44:14.366817. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


Steps
  SafarAnalyze............. out_files = wf.SafarAnalyze(cp, in_dir[, analyzer])
  SafarStem................ out_files = wf.SafarStem(cp, in_dir[, stemmer])
  align.................... changes, metadata = wf.align(file1, file2[, out_dir])
  apachetika............... out_files = wf.apachetika(in_files[, tika_server])
  basic-text-statistics.... metadata_out = wf.basic_text_statistics(in_files, out_file)
  clear-xml-elements....... out_file = wf.clear_xml_elements(element, xml_file)
  copy-and-rename.......... copy = wf.copy_and_rename(in_file[, rename])
  filter-nes............... filtered_nerstats = wf.filter_nes(nerstats[, name])
  flatten-dirs............. out = wf.flatten_dirs(in_dirs)
  freqs.................... freqs = wf.freqs(in_files[, name])
  frog-dir................. frogout = wf.frog_dir(in_dir[, skip])
  frog-single-text......... frogout = wf.frog_single_text(in_file)
  frog-to-saf.............. saf = wf.frog_to_saf(in_files)
  gather-dirs.............. out = wf.gathe

In [4]:
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    wf.load(step_file='https://raw.githubusercontent.com/arabic-digital-humanities/BlackLabIndexer-docker/master/blacklabindexer.cwl')
    
    print(wf.list_steps())
    
    analyzer = wf.add_input(analyzer='string', default='Alkhalil')
    book = wf.add_input(book='File')
    cp = wf.add_input(cp='string')
    index_name = wf.add_input(index_name='string', default='corpus')
    action = wf.add_input(action='string', default='create')
    index_format = wf.add_input(index_format='string', default='safar-analyzer')
    text_direction = wf.add_input(text_direction='string', default='rtl')
    content_viewable = wf.add_input(content_viewable='boolean', default=True)
    
    safar_output_dir = wf.safar_workflow(analyzer=analyzer, book=book, cp=cp) #, scatter='book'
    indexed = wf.blacklabindexer(action=action, 
                                 index_format=index_format, 
                                 index_name=index_name, 
                                 in_dir=safar_output_dir, 
                                 text_direction=text_direction, 
                                 content_viewable=content_viewable)
    
    wf.add_outputs(indexed=indexed)
    
    wf.add_outputs(safar_output_dir=safar_output_dir)
    
    wf.save('../adhtools/cwl/process.cwl', wd=True, relative=False)
    
    # select txt files -> ls?
    # divide and extract metadata -> safar2text_and_metadata
    # save to dir -> save_to_dir
    # safar analyze -> SafarAnalyze.java -> maak dat ie uit dir leest
    # add metadata -> safar_add_metadata
    # index with blacklab

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-03-27 10:47:18.102992. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


Steps
  SafarAnalyze............. out_files = wf.SafarAnalyze(cp, in_dir[, analyzer])
  SafarStem................ out_files = wf.SafarStem(cp, in_dir[, stemmer])
  align.................... changes, metadata = wf.align(file1, file2[, out_dir])
  apachetika............... out_files = wf.apachetika(in_files[, tika_server])
  basic-text-statistics.... metadata_out = wf.basic_text_statistics(in_files, out_file)
  blacklabindexer.......... out_dir = wf.blacklabindexer(action, in_dir, index_format, index_name[, contentViewable, textDirection])
  clear-xml-elements....... out_file = wf.clear_xml_elements(element, xml_file)
  copy-and-rename.......... copy = wf.copy_and_rename(in_file[, rename])
  filter-nes............... filtered_nerstats = wf.filter_nes(nerstats[, name])
  flatten-dirs............. out = wf.flatten_dirs(in_dirs)
  freqs.................... freqs = wf.freqs(in_files[, name])
  frog-dir................. frogout = wf.frog_dir(in_dir[, skip])
  frog-single-text......... frogout

In [4]:
# Now the scattered version
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    wf.load(step_file='https://raw.githubusercontent.com/arabic-digital-humanities/BlackLabIndexer-docker/master/blacklabindexer.cwl')
    
    print(wf.list_steps())
    
    analyzer = wf.add_input(analyzer='string', default='Alkhalil')
    in_dir = wf.add_input(in_dir='Directory')
    cp = wf.add_input(cp='string')
    index_name = wf.add_input(index_name='string', default='corpus')
    action = wf.add_input(action='string', default='create')
    index_format = wf.add_input(index_format='string', default='safar-analyzer')
    text_direction = wf.add_input(text_direction='string', default='rtl')
    content_viewable = wf.add_input(content_viewable='boolean', default=True)
    
    books = wf.ls(in_dir=in_dir)
    safar_output_dirs = wf.safar_workflow(analyzer=analyzer, book=books, cp=cp, scatter='book', scatter_method='dotproduct')
    merged_dir = wf.gather_dirs(in_dirs=safar_output_dirs)
    indexed = wf.blacklabindexer(action=action, 
                                 index_format=index_format, 
                                 index_name=index_name, 
                                 in_dir=merged_dir, 
                                 text_direction=text_direction, 
                                 content_viewable=content_viewable)
    wf.add_outputs(indexed=indexed)
    
    wf.add_outputs(safar_output_dirs=safar_output_dirs)
    wf.add_outputs(merged_dir=merged_dir)
    
    wf.save('../adhtools/cwl/process_dir.cwl', wd=True, relative=False)

/home/dafne/anaconda2/envs/adh/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-03-26 14:56:09.262912. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


Steps
  SafarAnalyze............. out_files = wf.SafarAnalyze(cp, in_dir[, analyzer])
  SafarStem................ out_files = wf.SafarStem(cp, in_dir[, stemmer])
  align.................... changes, metadata = wf.align(file1, file2[, out_dir])
  apachetika............... out_files = wf.apachetika(in_files[, tika_server])
  basic-text-statistics.... metadata_out = wf.basic_text_statistics(in_files, out_file)
  blacklabindexer.......... out_dir = wf.blacklabindexer(action, in_dir, index_format, index_name)
  clear-xml-elements....... out_file = wf.clear_xml_elements(element, xml_file)
  copy-and-rename.......... copy = wf.copy_and_rename(in_file[, rename])
  filter-nes............... filtered_nerstats = wf.filter_nes(nerstats[, name])
  flatten-dirs............. out = wf.flatten_dirs(in_dirs)
  freqs.................... freqs = wf.freqs(in_files[, name])
  frog-dir................. frogout = wf.frog_dir(in_dir[, skip])
  frog-single-text......... frogout = wf.frog_single_text(in_file)
  